In [20]:
import os
from pymongo import MongoClient
import re

PATH = os.getcwd()

In [5]:
client = MongoClient("mongodb://10.0.0.138:27017/")
db = client["rasa"]
collection = db["training_data"] 

In [49]:
# Write NLU YML:
f1 = open(PATH + "/nlu.yml", "w")
f2 = open(PATH  + "/domain.yml", "w")
f3 = open(PATH + "/rules.yml", "w")

f1.write("version: '3.1' \nnlu: \n")
f1.write("\n- intent: " + "xin_chao" + "\n" + "  examples: |\n")
line = """    - Xin chào
    - Hello
    - Chào bạn
    - Hi
    """
f1.write(line)
f1.write("\n- intent: " + "cam_on" + "\n" + "  examples: |\n")
line = """    - Cảm ơn bạn
    - Thank you\n
    """
f1.write(line)

f2.write("version: '3.1' \n")
f2.write("intents:\n")
f2.write("- xin_chao \n- cam_on\n\n")

f3.write("version: '3.1' \n")
f3.write("rules:\n")
f3.write("""- rule: rule_xin_chao
  steps:
  - intent: xin_chao
  - action: utter_xin_chao\n\n""")

items = collection.find()
for item in items:
    # print(item["_id"])
    # print(item["examples"])
    # print(item["response"])

    line = "\n- intent: " + "intent_" + str(item["_id"]) + "\n" + "  examples: |\n"
    f1.write(line)
    for example in item["examples"]:
        # new_example = re.sub(r'[^\w\s]', '', example)
        new_example = example.replace("\n", " ")
        line = "    - " + str(new_example) + "\n"
        f1.write(line)

    line = "- " + "intent_" + str(item["_id"]) + "\n"
    f2.write(line)

    line = "\n- rule: " + "rule_" + str(item["_id"]) + "\n  steps:"
    f3.write(line)
    line = "\n  - intent: intent_" + str(item["_id"]) 
    f3.write(line)
    line = "\n  - action: utter_" + str(item["_id"]) + "\n"
    f3.write(line)

f2.write("\n\nresponses:\n")
f2.write("  utter_xin_chao:\n")
f2.write("    - text: \"Xin chào bạn.\"\n\n")
f2.write("  utter_default:\n")
f2.write("    - text: \"Tôi xin lỗi, Tôi chưa có thông tin để trả lời bạn.\"\n\n")
items = collection.find()
for item in items:
    line = "  utter_" + str(item["_id"]) + ":\n" + "    - text: " + "\"" + item["response"] + "\"" + "\n" + "\n"
    f2.write(line)



f1.close()
f2.close()
f3.close()